# Description

Runs S-MultiXcan on a set of traits.

# Modules

In [1]:
import os
from glob import glob
from pathlib import Path

from utils import read_log_file_and_check_line_exists
import conf

# Settings

Apparently, there is no easy way to get the parent directory of
a notebook in Jupyter, so here I get that information either from
the parameter sent by `nbs/run_nbs.sh` (if called from command-line) or
from `os.getcwd()` (if called from browser).

In [2]:
PHENOPLIER_NOTEBOOK_FILEPATH = None
PHENOPLIER_NOTEBOOK_DIR = os.getcwd()

In [3]:
# Parameters
PHENOPLIER_NOTEBOOK_FILEPATH = (
    "projects/asthma-copd/nbs/15_twas/05-run_smultixcan.ipynb"
)

In [4]:
if PHENOPLIER_NOTEBOOK_FILEPATH is not None:
    PHENOPLIER_NOTEBOOK_DIR = str(Path(PHENOPLIER_NOTEBOOK_FILEPATH).parent)

display(PHENOPLIER_NOTEBOOK_DIR)

'projects/asthma-copd/nbs/15_twas'

In [5]:
SPREDIXCAN_DIR = conf.PROJECTS["ASTHMA_COPD"]["RESULTS_DIR"] / "twas" / "spredixcan"
display(SPREDIXCAN_DIR)
assert SPREDIXCAN_DIR.exists()

SPREDIXCAN_DIR_STR = str(SPREDIXCAN_DIR)
display(SPREDIXCAN_DIR_STR)

PosixPath('/opt/data/projects/asthma-copd/results/twas/spredixcan')

'/opt/data/projects/asthma-copd/results/twas/spredixcan'

In [6]:
OUTPUT_DIR = conf.PROJECTS["ASTHMA_COPD"]["RESULTS_DIR"] / "twas" / "smultixcan"
display(OUTPUT_DIR)

OUTPUT_DIR_STR = str(OUTPUT_DIR)
display(OUTPUT_DIR_STR)

PosixPath('/opt/data/projects/asthma-copd/results/twas/smultixcan')

'/opt/data/projects/asthma-copd/results/twas/smultixcan'

# Run

In [7]:
%%bash -s "$PHENOPLIER_NOTEBOOK_DIR" "$SPREDIXCAN_DIR_STR" "$OUTPUT_DIR_STR"
set -euo pipefail
# IFS=$'\n\t'

# read the notebook directory parameter and remove $1
export PHENOPLIER_NOTEBOOK_DIR="${PHENOPLIER_CODE_DIR}/$1"
shift

# read S-PrediXcan input dir
export SPREDIXCAN_DIR="$1"
shift

# read output dir
export OUTPUT_DIR="$1"
shift

run_job () {
    # run_job is a standard function name that performs a particular job
    # depending on the context. It will be called by GNU Parallel below.
    #
    # The implementation here runs the GLS model of PhenoPLIER on a trait.

    # read trait information
    # the first parameter to this function is a string with values separated by
    # commas (,). So here I split those into different variables.
    IFS=',' read -r pheno_id file sample_size n_cases <<< "$1"

    INPUT_FILENAME=${file%.*}

    # get input GWAS file, there should be a single file
    # here I make sure that there are no other files in the folder that
    # match this phenotype/trait filename prefix
    GWAS_DIR="${PHENOPLIER_PROJECTS_ASTHMA_COPD_RESULTS_DIR}/final_imputed_gwas"
    N_GWAS_FILES=$(ls ${GWAS_DIR}/${INPUT_FILENAME}*.txt.gz | wc -l)
    if [ "${N_GWAS_FILES}" != "1" ]; then
        echo "ERROR: found ${N_GWAS_FILES} GWAS files instead of one"
        exit 1
    fi
    INPUT_GWAS_FILEPATH=$(ls ${GWAS_DIR}/${INPUT_FILENAME}*.txt.gz)

    mkdir -p "${OUTPUT_DIR}"

    # make sure we are not also parallelizing within numpy, etc
    export NUMBA_NUM_THREADS=1
    export MKL_NUM_THREADS=1
    export OPEN_BLAS_NUM_THREADS=1
    export NUMEXPR_NUM_THREADS=1
    export OMP_NUM_THREADS=1

    echo "Running for $pheno_id"
    echo "Saving results in ${OUTPUT_DIR}"

    bash "${PHENOPLIER_CODE_DIR}/scripts/twas_smultixcan.sh" \
        --input-gwas-file "${INPUT_GWAS_FILEPATH}" \
        --spredixcan-folder "${SPREDIXCAN_DIR}" \
        --phenotype-name "${INPUT_FILENAME}" \
        --output-dir "${OUTPUT_DIR}"

    # print errors here in the notebook
    # first, look for the log file for this trait
    pattern="${OUTPUT_DIR}/${INPUT_FILENAME}*-gtex_v8-mashr-smultixcan*.log"

    N_LOG_FILES=$(ls ${pattern} | wc -l)
    if [ "${N_LOG_FILES}" != "1" ]; then
        echo "ERROR: found ${N_LOG_FILES} log files instead of one"
        exit 1
    fi
    LOG_FILE=$(ls ${pattern})

    cat "${LOG_FILE}" | grep -iE "warning|error"

    echo
}

# export function so GNU Parallel can see it
export -f run_job

# generate a list of run_job calls for GNU Parallel
# here I read a file with information about traits (one trait per line)
while IFS= read -r line; do
    echo run_job "${line}"
done < <(tail -n "+2" "${PHENOPLIER_PROJECTS_ASTHMA_COPD_TRAITS_INFO_FILE}") |
    parallel -k --lb --halt 2 -j${PHENOPLIER_GENERAL_N_JOBS}

Running for asthma_only
Saving results in /opt/data/projects/asthma-copd/results/twas/smultixcan


+ /opt/data/software/conda_envs/metaxcan/bin/python /opt/data/software/metaxcan/software/SMulTiXcan.py --models_folder /opt/data/data/phenomexcan/prediction_models/mashr --models_name_pattern 'mashr_(.*).db' --snp_covariance /opt/data/data/phenomexcan/prediction_models/gtex_v8_expression_mashr_snp_covariance.txt.gz --metaxcan_folder /opt/data/projects/asthma-copd/results/twas/spredixcan --metaxcan_filter 'GWAS_Asthma_only_GLM_SNPs_info0.7\-.*csv' --metaxcan_file_name_parse_pattern '(.*)\-gtex_v8\-mashr\-(.*).csv' --gwas_file /opt/data/projects/asthma-copd/results/final_imputed_gwas/GWAS_Asthma_only_GLM_SNPs_info0.7-harmonized-imputed.txt.gz --snp_column panel_variant_id --effect_allele_column effect_allele --non_effect_allele_column non_effect_allele --zscore_column zscore --keep_non_rsid --model_db_snp_key varID --cutoff_condition_number 30 --verbosity 7 --throw --output /opt/data/projects/asthma-copd/results/twas/smultixcan/GWAS_Asthma_only_GLM_SNPs_info0.7-gtex_v8-mashr-smultixcan.t

WARNING - Some GWAS snp zscores are not finite.



+ set +x


Running for copd_only
Saving results in /opt/data/projects/asthma-copd/results/twas/smultixcan


+ /opt/data/software/conda_envs/metaxcan/bin/python /opt/data/software/metaxcan/software/SMulTiXcan.py --models_folder /opt/data/data/phenomexcan/prediction_models/mashr --models_name_pattern 'mashr_(.*).db' --snp_covariance /opt/data/data/phenomexcan/prediction_models/gtex_v8_expression_mashr_snp_covariance.txt.gz --metaxcan_folder /opt/data/projects/asthma-copd/results/twas/spredixcan --metaxcan_filter 'GWAS_COPD_only_GLM_SNPs_info0.7\-.*csv' --metaxcan_file_name_parse_pattern '(.*)\-gtex_v8\-mashr\-(.*).csv' --gwas_file /opt/data/projects/asthma-copd/results/final_imputed_gwas/GWAS_COPD_only_GLM_SNPs_info0.7-harmonized-imputed.txt.gz --snp_column panel_variant_id --effect_allele_column effect_allele --non_effect_allele_column non_effect_allele --zscore_column zscore --keep_non_rsid --model_db_snp_key varID --cutoff_condition_number 30 --verbosity 7 --throw --output /opt/data/projects/asthma-copd/results/twas/smultixcan/GWAS_COPD_only_GLM_SNPs_info0.7-gtex_v8-mashr-smultixcan.txt


WARNING - Some GWAS snp zscores are not finite.



+ set +x


Running for aco
Saving results in /opt/data/projects/asthma-copd/results/twas/smultixcan
WARNING - Some GWAS snp zscores are not finite.



+ /opt/data/software/conda_envs/metaxcan/bin/python /opt/data/software/metaxcan/software/SMulTiXcan.py --models_folder /opt/data/data/phenomexcan/prediction_models/mashr --models_name_pattern 'mashr_(.*).db' --snp_covariance /opt/data/data/phenomexcan/prediction_models/gtex_v8_expression_mashr_snp_covariance.txt.gz --metaxcan_folder /opt/data/projects/asthma-copd/results/twas/spredixcan --metaxcan_filter 'GWAS_ACO_GLM_SNPs_info0.7\-.*csv' --metaxcan_file_name_parse_pattern '(.*)\-gtex_v8\-mashr\-(.*).csv' --gwas_file /opt/data/projects/asthma-copd/results/final_imputed_gwas/GWAS_ACO_GLM_SNPs_info0.7-harmonized-imputed.txt.gz --snp_column panel_variant_id --effect_allele_column effect_allele --non_effect_allele_column non_effect_allele --zscore_column zscore --keep_non_rsid --model_db_snp_key varID --cutoff_condition_number 30 --verbosity 7 --throw --output /opt/data/projects/asthma-copd/results/twas/smultixcan/GWAS_ACO_GLM_SNPs_info0.7-gtex_v8-mashr-smultixcan.txt
+ set +x


# Perform some checks in output and log files

In [8]:
assert OUTPUT_DIR.exists()

In [9]:
log_files = OUTPUT_DIR.glob("*.log")

In [10]:
for f in log_files:
    read_log_file_and_check_line_exists(
        f,
        [
            "INFO - Ran multi tissue in",
        ],
    )